# Schaefer Based Control Loop

In [360]:
import pandas as pd
import numpy as np
import plotly.express as px

from scipy.stats import norm
from scipy.optimize import minimize
from IPython.display import clear_output
from tqdm import tqdm

## The Data

In [361]:
df = pd.read_csv('rocklobster.csv')
df.head()

,year,catch,effort
0,1945,809,232
1,1946,854,253
2,1947,919,289
3,1948,1360,382
4,1949,1872,1046


In [362]:
px.line(df, x='year', y='catch', title='Catch (t)')

In [363]:
px.line(df, x='year', y='effort', title='Effort (1000 potlifts)')

In [364]:
df['cpue'] = df['catch']/df['effort']
px.line(df, x='year', y='cpue', title='CPUE (t/1000 potlift)')

## Fitting an Assessment

This is the key component of our control loop - getting an assessment and computing management reference points.

We're going to be doing a maximum likelihood estimation where we fit to the catch data. 

In [365]:
def schaefer_biomass(K, r, p, catch):
    B = [K*p]
    for i in range(len(catch)-1):
        B.append(B[i] + r*B[i]*(1-B[i]/K) - catch[i])
    return np.array(B)

def schaefer_catch(B, q, effort):
    return B*q*effort

def schaefer_NLL(params, catch, effort):
    K, r, p, q, scale = params
    B = schaefer_biomass(K, r, p, catch)
    catch_pred = schaefer_catch(B, q, effort)
    return -np.sum(norm.logpdf(np.log(catch), loc=np.log(catch_pred), scale=scale))

# these parameters come from my homework from class
def fit_schaefer(starting_params, catch, effort):
    return minimize(
        schaefer_NLL, 
        starting_params, 
        args=(catch, effort), 
        method='Nelder-Mead',
        bounds=((1, float('inf')), (0.0, 1), (1, 1), (0, 1), (0, float('inf')))
    ).x

def EMSY(params):
    _, r, _, q, _ = params
    return r/(2 * q)

starting_params = [145580.65165869, 0.0467626394908834, 1, 0.0000216697659417997, 100]
params = fit_schaefer(starting_params, df['catch'], df['effort'])
REAL_PARAMS = params
print(EMSY(params))


/usr/local/lib/python3.10/dist-packages/scipy/stats/_distn_infrastructure.py:2024: RuntimeWarning:

divide by zero encountered in divide



1077.7765687462213


In [366]:
def project_forward(real_params, historic_df, EMSY, years):
    K, r, p, q, scale = real_params
    B = schaefer_biomass(K, r, p, historic_df['catch'])[-1]
    catch = historic_df['catch'].values[-1] 
    B = B + r*B*(1-B/K) - catch
    projected_catch = []
    projected_years = []
    projected_effort = []
    projected_cpue = []
    current_year = historic_df['year'].max()
    projected_biomass = []
    for i in range(years):
        projected_biomass.append(B)
        catch = B * q * EMSY 
        log_catch_w_error = norm.rvs(loc=np.log(catch), scale=scale, size=1)[0]
        catch = np.exp(log_catch_w_error)
        B = B + r*B*(1-B/K) - catch
        projected_catch.append(catch)
        projected_years.append(current_year + i + 1)
        projected_effort.append(EMSY)
        projected_cpue.append(catch/EMSY)

    return pd.DataFrame({
        'year': projected_years,
        'catch': projected_catch,
        'effort': projected_effort,
        'cpue': projected_cpue,
        'biomass': projected_biomass
    })

In [367]:
def BMSY(params):
    K, *_ = params
    return K/2

def run_loop(steps, interim_years, starting_params, df, real_params):
    pdf = df.copy()
    params = starting_params
    for _ in range(steps):
        params = fit_schaefer(params, pdf['catch'], pdf['effort'])
        clear_output()
        effort = EMSY(params)
        K, r, p, q, scale = params
        B = schaefer_biomass(K, r, p, pdf['catch'])[-1]
        if B < BMSY(params):
            effort = effort * B/BMSY(params)
        pdf = pd.concat([pdf, project_forward(real_params, pdf, effort, interim_years)]).reset_index(drop=True)
    return pdf
total_years = 100
interim_years = 5
steps = int(total_years/interim_years)
params = REAL_PARAMS
pdfs = []
runs = 10
for i in range(runs):
    pdf = run_loop(steps, interim_years, params, df, params)
    pdf['run'] = i
    pdfs.append(pdf)
pdf = pd.concat(pdfs).reset_index(drop=True)
px.line(pdf[pdf['year'] > 1990], x='year', y='catch', color='run', title='Catch')
    

In [368]:
catch_error = pdf.groupby('year').agg({'catch': ('std', 'mean')})['catch'].reset_index()
catch_error['error'] = catch_error['std'] * 1.96
px.line(catch_error[catch_error['year'] > 1990], x='year', y='mean', error_y='error', title='Catch Error')

In [369]:
px.line(pdf[pdf['year'] > 1990], x='year', y='biomass', color='run', title='Biomass')

## As a Class

We're going to assume that the thing that we have control over is the catch limits. We can specify how much folks should be able to take each year. What's varying is how much effort is required to take that catch. 

In [412]:
from functools import partial

def schaefer_biomass(params, catch):
    K, r, p, *_ = params
    B = [K*p]
    for i in range(len(catch)-1):
        B.append(B[i] + r*B[i]*(1-B[i]/K) - catch[i])
    return np.array(B)

def fox_biomass(params, catch):
    K, r, p, *_ = params
    B = [K*p]
    for i in range(len(catch)-1):
        B.append(B[i] + r*B[i]*(-np.log(B[i]/K)) - catch[i])
    return np.array(B)

def biomass_dynamics_catch(params, B, effort):
    *_, q, _ = params
    return B*q*effort

def biomass_dynamics_NLL(biomass_func, params, catch, effort):
    *_, scale = params
    B = biomass_func(params, catch)
    catch_pred = biomass_dynamics_catch(params, B, effort)
    return -np.sum(norm.logpdf(np.log(catch), loc=np.log(catch_pred), scale=scale))

# these parameters come from my homework from class
def fit_biomass_dynamics(biomass_func, starting_params, catch, effort):
    NLL = partial(biomass_dynamics_NLL, biomass_func)
    return minimize(
        NLL, 
        starting_params, 
        args=(catch, effort), 
        method='Nelder-Mead',
        bounds=((1, float('inf')), (0.0, 1), (1, 1), (0, 1), (0, float('inf')))
    ).x

class SchaeferStock(object):
    def __init__(self, B, K, r, q, K_error, r_error, q_error):
        self.B, self.K, self.r, self.q = B, K, r, q
        self.K_error, self.r_error, self.q_error = K_error, r_error, q_error
        self.log_q = np.log(q)
        self.reset()

    def reset(self):
        self.physical = {
            'biomass': [self.B],
            'K': [],
            'r': [],
            'q': []
        }
        self.measureable = {
            'catch': [],
            'effort': []
        }

    def draw_q(self):
        return np.exp(norm.rvs(loc=self.log_q, scale=self.q_error, size=1)[0])
    
    def draw_r(self):
        return self.r
    
    def draw_K(self):
        return self.K
    
    @staticmethod
    def update_biomass(B, r, K, catch):
        return B + r*B*(1-B/K) - catch

    def fish(self, catch_limit):
        K, r, q = self.draw_K(), self.draw_r(), self.draw_q()
        B = self.physical['biomass'][-1]

        catch = min(B*0.9, catch_limit)
        effort = catch/(q*B)
        B = self.update_biomass(B, r, K, catch)

        self.measureable['catch'].append(catch)
        self.measureable['effort'].append(effort)

        self.physical['K'].append(K)
        self.physical['r'].append(r)
        self.physical['q'].append(q)
        self.physical['biomass'].append(B)

class FoxStock(SchaeferStock):
    @staticmethod
    def update_biomass(B, r, K, catch):
        return B + r*B*(-np.log(B/K)) - catch

class UnmanagedFishery(object):
    def __init__(self, desired_catch, catch_error):
        self.desired_catch = desired_catch
        self.catch_error = catch_error

    def fish(self):
        return norm.rvs(loc=self.desired_catch, scale=self.catch_error, size=1)[0]


class SchaeferFishery(object):
    def __init__(self, starting_params, catch_error, catch_limit=0):
        self.starting_params = starting_params
        self.catch_limit = catch_limit
        self.catch_error = catch_error
        self.reset()

    def reset(self):
        self.params = self.starting_params   
        self.desired_catch = self.catch_limit     

    def manage(self, stock_measureables):
        self.params = fit_biomass_dynamics(schaefer_biomass, self.params, stock_measureables['catch'], stock_measureables['effort'])
        K, r, *_ = self.params
        self.desired_catch = K * r / 4

    def fish(self):
        return norm.rvs(loc=self.desired_catch, scale=self.catch_error, size=1)[0]
    
class FoxFishery(SchaeferFishery):
    def manage(self, stock_measureables):
        self.params = fit_biomass_dynamics(fox_biomass, self.params, stock_measureables['catch'], stock_measureables['effort'])
        K, r, *_ = self.params
        self.desired_catch = K * r / np.e
        

stock = SchaeferStock(200000, 200000, 0.1, 0.0001, 0, 0, 0.05)
fishery = UnmanagedFishery(10000, 100)
for _ in range(20):

    catch = fishery.fish()
    stock.fish(catch)

timeline = pd.DataFrame({
    'effort': stock.measureable['effort'],
    'catch': stock.measureable['catch'],
    'year': range(len(stock.measureable['catch']))
})
timeline['cpue'] = timeline['catch'] / timeline['effort']

px.line(timeline, x='year', y='cpue')

In [413]:
stock = SchaeferStock(200000, 200000, 0.1, 0.0001, 0, 0, 0.05)
fishery = UnmanagedFishery(10000, 100)
for _ in range(10):
    catch = fishery.fish()
    stock.fish(catch)

fishery = SchaeferFishery([200000, 0.1, 0.0001, 1, 100], 100, 10000)
fishery.manage(stock.measureable)
for _ in range(50):
    catch = fishery.fish()
    stock.fish(catch)

timeline = pd.DataFrame({
    'effort': stock.measureable['effort'],
    'catch': stock.measureable['catch'],
    'year': range(len(stock.measureable['catch']))
})
timeline['cpue'] = timeline['catch'] / timeline['effort']

px.line(timeline, x='year', y='cpue')

/tmp/ipykernel_2985/2009648455.py:30: OptimizeWarning:

Initial guess is not within the specified bounds

/usr/local/lib/python3.10/dist-packages/scipy/stats/_distn_infrastructure.py:2024: RuntimeWarning:

divide by zero encountered in divide

/tmp/ipykernel_2985/2009648455.py:25: RuntimeWarning:

invalid value encountered in log



In [414]:
stock = FoxStock(200000, 200000, 0.1, 0.0001, 0, 0, 0.05)
fishery = UnmanagedFishery(10000, 100)
for _ in range(10):
    catch = fishery.fish()
    stock.fish(catch)

fishery = FoxFishery([200000, 0.1, 0.0001, 1, 100], 100, 10000)
fishery.manage(stock.measureable)
for _ in range(50):
    catch = fishery.fish()
    stock.fish(catch)

timeline = pd.DataFrame({
    'effort': stock.measureable['effort'],
    'catch': stock.measureable['catch'],
    'year': range(len(stock.measureable['catch']))
})
timeline['cpue'] = timeline['catch'] / timeline['effort']

px.line(timeline, x='year', y='cpue')

/tmp/ipykernel_2985/2009648455.py:30: OptimizeWarning:

Initial guess is not within the specified bounds

/usr/local/lib/python3.10/dist-packages/scipy/stats/_distn_infrastructure.py:2024: RuntimeWarning:

divide by zero encountered in divide

/tmp/ipykernel_2985/2009648455.py:14: RuntimeWarning:

invalid value encountered in log

/tmp/ipykernel_2985/2009648455.py:25: RuntimeWarning:

invalid value encountered in log



In [415]:
fishery.desired_catch

6216.792427136958